# Coding Assignment

1. Read the input testfile (Pipe delimited) provided as a "Spark RDD" 
2. Remove the Header Record from the RDD
3. Calculate Final_Price:
        Final_Price = (Size * Price_SQ_FT)
4. Save the final rdd into Textfile with
        Property_ID|Location|Final_Price

In [1]:
import findspark
findspark.init()

In [2]:
# Create SparkSession and sparkcontext
from pyspark.sql import SparkSession
spark = SparkSession.builder\
                    .master("local")\
                    .appName('Assignment 2')\
                    .getOrCreate()
sc=spark.sparkContext

In [77]:
#Read the input file as RDD using Spark Context
rdd_in=sc.textFile("input.txt")

#Apply Filter to get header and data
rdd1=rdd_in.filter(lambda l: not l.startswith("Property_ID"))
header=rdd_in.filter(lambda l: l.startswith("Property_ID"))

#Apply Map and flatMap to get the column data
rdd2=rdd1.flatMap(lambda x:x.split(',')).map(lambda x: x.split('|'))

In [68]:
#Get the index of the required column
col_list=header.first().split('|')
f1=col_list.index("Property_ID")
f2=col_list.index("Location")
f3=col_list.index("Size")
f4=col_list.index("Price_SQ_FT")

In [70]:
#Function definition to calculate the final price
def mul_price(d1,d2):
    res=float(d1)*float(d2)
    return str(res)

In [80]:
#Call the function and create final result as expected
header_out=header.map(lambda x: x.split("|")[f1]+"|"+x.split("|")[f2]+"|Final_Price")
rdd3=rdd2.map(lambda x: x[f1]+"|"+x[f2]+"|"+ mul_price(x[f3],x[f4]))

final_out=header_out.union(rdd3)

In [85]:
final_out.collect()

['Property_ID|Location|Final_Price',
 '1461262|Arroyo Grande|866126.3',
 '1478004|Paulo Pablo|460996.62',
 '1486551|Paulo Pablo|545002.0',
 '1492832|Santa Bay|1015201.2',
 '1499102|Thomas Country|123638.51']

In [88]:
#Save the final Spark RDD as textfile
final_out.coalesce(1).saveAsTextFile("output.txt")